# Gallery Example: M/Hyper/1 Tandem Network

This example demonstrates a simple 2-queue tandem network:
- **Arrivals**: Exponential (Poisson process)
- **Service**: Hyperexponential at both queues
- **Servers**: 1 server per queue
- **Scheduling**: FCFS

This is a special case of the linear network with n=2.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_mhyp1_tandem(): Network {
    """Create M/Hyp/1-Tandem queueing network (2 queues in series)"""
    val model = Network("M/Hyp/1-Tandem")
// Block 1: nodes
    val source = Source(model, "mySource")
    val queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)
    val queue2 = Queue(model, "Queue2", SchedStrategy.FCFS)
    val sink = Sink(model, "mySink")
// Block 2: classes
    val oclass = OpenClass(model, "myClass")
// Exponential arrivals
    source.setArrival(oclass, Exp(1))
// HyperExp service with SCV=2 at both queues
    queue1.setService(oclass, HyperExp.fit_mean_and_scv(0.1, 2))
    queue2.setService(oclass, HyperExp.fit_mean_and_scv(0.1, 2))
// Block 3: topology
    model.link(Network.serialRouting(source, queue1, queue2, sink))
    
    return model
// Create the model
val model = gallery_mhyp1_tandem()

## About M/Hyper/1 Tandem

This 2-queue tandem model has high service variability (SCV = 2) at both queues.

Key characteristics:
- Poisson arrivals provide baseline randomness
- High service variability amplifies queue effects
- The departure process from Queue 1 is smoother than its arrival
- Queue 2 may have different behavior due to modified input process

This demonstrates how variability propagates through a tandem network.

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)